# Ch16. 합성곱 신경망 (Convolution Neural Network)

# v01. 합성곱 신경망 (Convolution Neural Network)

- 합성곱 신경망(Convolutional Neural Network)은 이미지 처리에 탁월한 성능을 보이는 신경망이다.
- 이 책의 주제는 이미지 처리가 아니라 자연어 처리이기 때문에, 어떻게 보면 책의 범위를 벗어났다고 느낄 수도 있다.
- 하지만 합성곱 신경망으로도 자연어 처리를 할 수 있다.
- 합성곱 신경망을 이용한 자연어 처리를 배우기 전에 합성곱 신경망에 대해서 먼저 이해해보자.
- 이번 챕터에 한해서만 현재 이미지 처리를 배우고 있다고 가정하고 설명한다.

<br>

- 합성곱 신경망은 크게 **합성곱층(Convolution layer)**과 **풀링층(Pooling layer)**으로 구성된다.
- 아래의 그림은 합성곱 신경망의 일반적인 예를 보여준다.

$\quad$ ![](https://wikidocs.net/images/page/64066/convpooling.PNG)

- [http://cs231n.github.io/convolutional-networks](http://cs231n.github.io/convolutional-networks)

- 위의 그림에서 `CONV`는 합성곱 연산을 의미하고, 합성곱 연산의 결과가 활성화 함수 `ReLU`를 지난다.
- 이 두 과정을 합성곱층이라고 한다.

- 그 후에 `POOL`이라는 구간을 지나는 데 이는 풀링 연산을 의미하며 풀링층이라고 한다.

<br>

- 이번 챕터에서는 합성곱 연산과 풀링 연산의 의미에 대해서 학습한다.

<br>

## 1.1 합성곱 신경망의 대두

- 합성곱 신경망은 이미지 처리에 탁월한 성능을 보이는 신경망이다.

<br>

### 1.1.1 이미지 처리

- 이미지 처리를 하기 위해서 앞서 배운 다층 퍼셉트론을 사용할 수 있지만 한계가 있었다.
- 예를 들어, 알파벳 손글씨를 분류하는 어떤 문제가 있다고 해보자.
- 아래의 그림은 알파벳 Y를비교적 정자로 쓴 손글씨와 다소 휘갈겨 쓴 손글씨 두 개를 2차원 텐서인 행렬로 표현한 것이다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv0.png)

- 사람이 보기에는 두 그림 모두 알파벳 Y로 손쉽게 판단이 가능하다.
- 하지만 기계가 보기에는 각 픽셀마다 가진 값이 거의 상이하므로 완전히 다른 값을 가진 입력이다.

- 그런데 이미지라는 것은 위와 같이 같은 대상이라도 휘어지거나, 이동되었거나, 방향이 뒤틀렸거나 등 다양한 변형이 존재한다.
- 다층 퍼셉트론은 몇 가지 픽셀만 가밧이 달라져도 민감하게 예측에 영향을 받는다는 단점이 있다.

<br>

- 좀 더 구체적으로 살펴보자.
- 위 손글씨를 다층 퍼셉트론으로 분류한다고 하면, 이미지를 1차원 텐서인 벡터로 변환하고 다층 퍼셉트론의 입력층으로 사용해야 한다.
- 두 번째 손글씨를 다층 퍼셉트론으로 분류하기 위해서 벡터로 바꾸면 다음과 같다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv1.png)

- 1차원으로 변환된 결과는 사람이 보기에도 이게 원래 어떤 이미지였는 지 알아보기가 어렵다.
- 이는 기계도 마찬가지이다.

<br>

### 1.1.2 공간적인 구조(spatial structure) 정보

- 위와 같이 결과는 변환 전에 가지고 있던 **공간적인 구조(spatial structure) 정보**가 유실된 상태이다.
- 여기서 공간적인 구조 정보라는 것은 거리가 가까운 어떤 픽셀들끼리는 어떤 연관이 있고, 어떠 픽셀들끼리는 값이 비슷하거나 등을 포함하고 있다.
- 결국 이미지의 공간적인 구조 정보를 보존하면서 학습할 수 있는 방법이 필요해졌다.
- 이를 위해 사용하는 것이 합성곱 신경망이다.

<br>

## 1.2 채널 (Channel)

- 이미지 처리의 기본적인 용어인 채널에 대해서 간단히 정의한다.

<br>

### 1.2.1 이미지의 텐서 종류

- 기계는 글자나 이미지보다 숫자, 다시 말해 텐서를 더 잘 처리할 수 있다.
- 이미지는 `(높이, 너비, 채널)`이라는 **3차원 텐서**이다.
  - `높이` : 이미지의 세로 방향 픽셀 수
  - `너비` : 이미지의 가로 방향 픽셀 수
  - `채널` : 색 성분

- 흑백 이미지는 채널 수가 1이다.
- 각 픽셀은 0부터 255 사이의 값을 가진다.

<br>

### 1.2.2 손글씨 데이터 (흑백 이미지)

- 아래는 28 x 28 픽셀의 손글씨 데이터를 보여준다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv2.png)

- 위의 손글씨 데이터는 흑백 이미지므로 채널수가 1임을 고려하면 `(28 x 28 x 1)`의 크기를 가지는 3차원 텐서이다.

<br>

### 1.2.3 컬러 이미지

- 그렇다면 흑백이 아니라 우리가 통상적으로 접하게 되는 컬러 이미지는 어떨까?
- 컬러 이미지는 **적색(Red)**, **녹색(Green)**, 청색(Blue)** 채널 수가 3개이다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv3.png)

- 하나의 픽셀은 세 가지 색깔, 삼원색의 조합으로 이루어진다.
- 만약, 높이가 28, 너비가 28인 컬러 이미지가 있다면 이 이미지의 텐서는 `(28 x 28 x 3)`의 크기를 가지는 3차원 텐서이다.
- 채널은 때로는 **깊이(depth)**라고도 한다.
- 이 경우 이미지는 `(높이, 너비, 깊이)`라는 3차원 텐서로 표현된다고 말할 수 있을 것이다.

<br>

## 1.3 합성곱 연산 (Convolution operation)

- 합성곱층은 합성곱 연산을 통해서 **이미지의 특징을 추출**하는 역할을 한다.
- 우선, 합성곱 연산에 대해서 이해해보자.

<br>

### 1.3.1 합성곱

- 합성곱은 영어로 컨볼루션(convolution)이라고도 불린다.
- **커널(kernel)** 또는 **필터(filter)**라는 $n \times m$ 크기의 행렬로 높이$(height)$ $\times$ 너비$(width)$ 크기의 이미지를 처음부터 끝까지 겹치며 훑으면서 $n \times m$ 크기의 겹쳐지는 부분의 각 이미지와 커널의 원소의 값을 곱해서 모두 더한 값을 출력으로 하는 것을 말한다.
- 이 때, 이미지의 가장 왼쪽 위부터 가장 오른쪽까지 순차적으로 훑는다.
- 커널(kernel)은 일반적으로 3 x 3 또는 5 x 5를 사용한다.

<br>

### 1.3.2 합성곱 연산 수행 과정

- 예를 통해 이해해보자.
- 아래는 $3 \times 3$ 크기의 커널로 $5 \times 5$의 이미지 행렬에 합성곱 연산을 수행하는 과정을 보여준다.
- 한 번의 연산을 1 스탭(step)이라고 했을 때, 합성곱 연산의 네 번째 스탭까지 이미지와 식으로 표현했다.

<br>

#### 1.3.2.1 첫 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv4.png)

- (1×1) + (2×0) + (3×1) + (2×1) + (1×0) + (0×1) + (3×0) + (0×1) + (1×0) = 6

<br>

#### 1.3.2.2 두 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv5.png)

- (2×1) + (3×0) + (4×1) + (1×1) + (0×0) + (1×1) + (0×0) + (1×1) + (1×0) = 9

<br>

#### 1.3.2.3 세 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv6.png)

- (3×1) + (4×0) + (5×1) + (0×1) + (1×0) + (2×1) + (1×0) + (1×1) + (0×0) = 11

<br>

#### 1.3.2.4 네 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv7.png)

- (2×1) + (1×0) + (0×1) + (3×1) + (0×0) + (1×1) + (1×0) + (4×1) + (1×0) = 10

<br>

#### 1.3.2.5 최종 결과

- 위 연산을 총 9번의 스탭가지 마쳤다고 가정하였을 때, 최종 결과는 아래와 같다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv8.png)

<br>

### 1.3.3 특성 맵 (feature map)

- 위와 같이 입력으로부터 커널을 사용하여 합성곱 연산을 통해 나온 결과를 **특성 맵(feature map)**이라고 한다.
- 위의 예제에서는 커널의 크기가 3 x 3 이였다.
- 하지만 커널의 크기는 사용자가 정할 수 있다.

<br>

### 1.3.4 스트라이드 (stride)

- 또한 커널의 이동 범위가 위의 예제에서는 한 칸이였다.
- 이 또한 사용자가 정할 수 있다.
- 이러한 이동 범위를 **스트라이드(stride)**라고 한다.

- 아래의 예제는 스트라이드가 2일 경우에 5 x 5 이미지에 합성곱 연산을 수행하는 3 x 3 커널의 움직임을 보여준다.
- 최종적으로 2 x 2의 크기의 특성 맵을 얻는다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv9.png)

<br>

## 1.4 패딩 (Padding)

- 위의 예제에서 5 x 5  5 x 5 이미지에 3 x 3의 커널로 합성곱 연산을 하였을 때, 스트라이드가 1일 경우에는 3 x 3의 특성 맵을 얻었다.
- 이와 같이 합성곱 연산의 결과로 얻은 특성 맵은 **입력보다 크기가 작아진다는 특징**이 있다.
- 만약, 합성곱 층을 여러 개 쌓았다면 최종적으로 얻은 특성 맵은 초기 입력보다 매우 작아진 상태가 되버린다.

- 합성곱 연산 이후에도 특성 맵의 크기가 입력의 크기와 동일하게 유지되도록 하고 싶다면 패딩(padding)을 사용하면 된다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv10.png)

- 패딩은 (합성곱 연산을 하기 전에) 입력의 가장자리에 지정된 개수의 폭만큼 행과 열을 추가해주는 것을 말한다.
- 좀 더 쉽게 설명하면 지정된 개수의 폭만큼 테두리를 추가한다.
- 주로 값을 0으로 채우는 **제로 패딩(zero padding)**을 사용한다.

- 위의 그림은 5 x 5 이미지에 1폭짜리 제로 패딩을 사용하여 위, 아래에 하나의 행을 좌, 우에 하나의 열을 추가한 모습을 보여준다.

<br>

- 커널은 주로 3 x 3 또는 5 x 5의 크기를 사용한다.
  - 스트라이드가 1이고 3 x 3 크기의 커널을 사용  
  $\rightarrow$ 1폭짜리 제로 패딩을 사용하면 입력과 특성 맵의 크기를 보존할 수 있다.
  - 스트라이드가 1이고 5 x 5 크기의 커널을 사용  
  $\rightarrow$ 2폭짜리 제로 패딩을 사용하면 입력과 특성 맵의 크기를 보존할 수 있다.

- 예를 들어 5 x 5 크기의 이미지에 1폭짜리 제로 패딩을 하면 7 x 7 이미지가 된다.
- 여기에 3 x 3의 커널을 사용하여 1 스트라이드로 합성곱을 한 후의 특성 맵은 기존의 입력 이미지의 크기와 같은 5 x 5가 된다.

<br>

## 1.5 가중치와 편향

- 합성곱 신경망에서의 가중치와 편향을 이해하기 위해서 우선 다층 퍼셉트론과 비교하여 살펴보자.

<br>

### 1.5.1 가중치

#### 1.5.1.1 다층 퍼셉트론의 가중치

- 다층 퍼셉트론으로 3 x 3 이미지를 처리한다고 가정해보자.
- 우선 이미지를 1차원 텐서인 벡터로 만들면, 3 x 3 = 9가 되므로 입력층은 9개의 뉴런을 가진다.
- 그리고 4개의 뉴런을 가지는 은닉층을 추가한다고 해보자.
- 이는 아래의 그림과 같다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv11.png)

- 위에서 각 연결선은 가중치를 의미한다.
- 위의 그림에서는 9 x 4 = 36개의 가중치를 가진다.

<br>

#### 1.5.1.2 합성곱 신경망의 가중치

- 이제 비교를 위해 합성곱 신경망으로 3 x 3 이미지를 처리한다고 해보자.
- 2 x 2 커널을 사용하고, 스트라이드는 1로 한다.
- `*`는 합성곱 연산을 의미한다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv12.png)

- 사실 합성곱 신경망에서 가중치는 커널 행렬의 원소들이다.
- 이를 인공 신경망의 형태로 표현한다면 다음과 같이 표현할 수 있다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv13.png)

- 최종적으로 특성 맵을 얻기 위해서는 동일한 커널로 이미지 전체르 훑으며 합성곱 연산을 진행한다.
- 결국 이미지 전체를 훑으면서 사용되는 가중치는 $w_0, w_1, w_2, w_3$ 4개 뿐이다.
- 그리고 각 합성곱 연산마다 이미지의 모든 픽셀을 사용하는 것이 아니라, 커널과 맵핑되는 픽셀만을 입력으로 사용하는 것을 볼 수 있다.
- 결국 합성곱 신경망은 다층 퍼셉트론을 사용할 때보다 훨씬 적은 수의 가중치를 사용하며 공간적 구조 정보를 보존한다는 특징이 있다.

<br>

### 1.5.2 활성화 함수

- 다층 퍼셉트론의 은닉층에서는 가중치 연산 후에 비선형성을 추가하기 위해서 활성화 함수를 통과시켰다.
- 합성곱 신경망의 은닉층에서도 마찬가지이다.
- 합성곱 연산을 통해 얻은 특성 맵은 다층 퍼셉트론때와 마찬가지로 비선형성 추가를 위해서 활성화 함수를 지나게 된다.
- 이떄 **렐루 함수**나 **렐루 함수의 변형들**이 주로 사용된다.
- 은닉층에서 렐루 함수가 주로 사용되는 이유는 앞서 활성화 함수 챕터에서 다뤘다.

<br>

### 1.5.3 합성곱 층 (convolution layer)

- 이와 같이 합성곱 연산을 통해서 특성 맵을 얻고, 활성화 함수를 지나는 연산을 하는 합성공 신경망의 은닉층을 합성곱 신경망에서는 **합성곱 층(convolution layer)**이라고 한다.

<br>

### 1.5.4 합성곱 신경망의 편향

$\quad$ ![](https://wikidocs.net/images/page/64066/conv14.png)

- 합성곱 신경망에도 편향(bias)를 당연히 추가할 수 있다.
- 만약, 편향을 사용한다면 커널을 적용한 뒤에 더해진다.
- 편향은 하나의 값만 존재하며, 커널이 적용된 결과의 모든 원소에 더해진다.

<br>

## 1.6 특성 맵의 크기 계산 방법

- 입력의 크기와 커널의 크기, 그리고 스트라이드의 값만 알면 합성곱 연산의 결과인 특성 맵의 크기를 계산할 수 있다.

<br>

### 1.6.1 용어 정의

- $I_h$ : 입력의 높이
- $I_w$ : 입력의 너비
- $K_h$ : 커널의 높이
- $K_w$ : 커널의 너비
- $S$ : 스트라이드
- $O_h$ : 특성 맵의 높이
- $O_w$ : 특성 맵의 너비

<br>

### 1.6.2 특성 맵의 높이와 너비 수식

- 위의 용어 정의에 따라 특성 맵의 높이와 너비는 다음과 같다.

$
\qquad
O_h = floor \left( \frac{I_h - K_h}{S} + 1 \right)
$

$
\qquad
O_w = floor \left( \frac{I_w - K_w}{S} + 1 \right)
$

- 여기서 $floor$ 함수는 소수점 발생 시 소수점 이하를 버리는 역할을 한다.
- 예를 들어, 5 x 5 크기의 이미지에 3 x 3 커널을 사용하고, 스트라이드 1로 합성곱 연산을 수행한다고 하자.
- 이 경우 특성 맵의 크기는 (5 - 3 + 1) x (5 - 3 + 1) = 3 x 3 임을 알 수 있다.
- 이는 또한 총 9번의 스텝이 필요함을 의미하기도 한다.

<br>

### 1.6.3 패딩의 폭을 포함한 특성 맵의 높이와 너비 수식

- 패딩의 폭을 $P$라고 하고, 패딩까지 고려한 식은 다음과 같다.

$
\qquad
O_h = floor \left( \frac{I_h - K_h + 2P}{S} + 1 \right)
$

$
\qquad
O_w = floor \left( \frac{I_w - K_w + 2P}{S} + 1 \right)
$

<br>

## 1.7 다수의 채널을 가질 경우의 합성곱 연산 (3차원 텐서의 합성곱 연산)

- 지그까지는 채널(channel) 또는 깊이(depth)를 고려하지 않고, 2차원 텐서를 가정하고 설명했다.
- 하지만 실제로 합성곱 연산의 입력은 **'다수의 채널을 가진' 이미지** 또는 이전 연산의 결과로 나온 특성 맵일 수 있다.
- 만약, 다수의 채널을 가진 입력 데이터를 가지고 합성곱 연산을 한다고 하면 커널의 채널 수도 입력의 채널 수 만큼 존재해야 한다.
- 다시 말해 입력 데이터의 채널 수와 커널의 채널 수는 같아야 한다.
- 채널 수가 같으므로 합성곱 연산을 채널마다 수행한다.
- 그리고 그 결과를 모두 더하여 최종 특성 맵을 얻는다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv15.png)

- 위 그림은 3개의 채널을 가진 입력 데이터와 3개의 채널을 가진 커널의 합성곱 연산을 보여준다.
- 커널의 각 채널끼리의 크기는 같아야 한다.

- 각 채널 간 합성곱 연산을 마치고, 그 결과를 모두 더해서 하나의 채널을 가지는 특성 맵을 만든다.
- 주의할 점은 위의 연산에서 사용되는 커널은 **3개의 커널이 아니라 3개의 채널을 가진 1개의 커널**이라는 점이다.

- 위 그림은 높이 3, 너비 3, 채널 3의 입력이 높이 2, 너비 2, 채널 3의 커널과 합성곱 연산을 하여 높이 2, 너비 2, 채널 1의 특성 맵을 얻는다는 의미이다.
- 합성곱 연산의 결과로 얻은 특성 맵의 채널 차원은 RGB 채널 등과 같은 컬러의 의미를 담고 있지는 않다.

<br>

- 이제 이 연산에서 각 차원을 변수로 두고 좀 더 일반화해보자.

<br>

## 1.8 3차원 텐서의 합성곱 연산

### 1.8.1 각 변수의 의미

- 일반화를 위해 사용하는 각 변수가 의미하는 바는 다음과 같다.

- $I_h$ : 입력의 높이
- $I_w$ : 입력의 너비
- $K_h$ : 커널의 높이
- $K_w$ : 커널의 너비
- $O_h$ : 특성 맵의 높이
- $O_w$ : 특성 맵의 너비
- $C_i$ : 입력 데이터의 채널

<br>

### 1.8.2 3차원 텐서의 합성곱 연산

$\quad$ ![](https://wikidocs.net/images/page/64066/conv16_final.png)

높이 $I_h$, 너비 $I_w$, 채널 $C_i$의 입력 데이터는 동일한 채널 수 $C_i$를 가지는 높이 $K_h$, 너비 $K_w$의 커널과 합성곱 연산을 하여 높이 $O_h$, 너비 $O_w$, 채널 $1$의 특성 맵을 얻습니다.

<br>

### 1.8.3 여러 개의 커널을 사용하는 합성곱 연산

- 그런데 하나의 입력에 여러 개의 커널을 사용하는 합성곱 연산을 수행할 수도 있다.
- 합성곱 연산에서 다수의 커널을 사용할 경우, 특성 맵의 크기가 어떻게 바뀌는 지 확인해보자.
- 다음은 $C_o$를 합성곱 연산에 사용하는 커널의 수라고 하였을 때의 합성곱 연산 과정을 보여준다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv17_final_final.PNG)

- 합성곱 연산에서 다수의 커널을 사용할 경우, 사용한 커널 수는 합성곱 연산의 결과로 나오는 특성 맵의 채널 수가 된다.

<br>

### 1.8.4 가중치 매개변수의 총 개수

- 위 내용을 이해했다면 커널의 크기와 입력 데이터의 채널 수 $C_i$와 특성 맵(출력 데이터)의 채널 수 $C_o$가 주어졌을 때, 가중치 매개변수의 총 개수를 구할 수 있다.

- 가중치는 커널의 원소들이다.
- 그러므로 하나의 커널의 하나의 채널은 $K_i \times K_o$개의 매개변수를 가지고 있다.

- 그런데 합성곱 연산을 하려면 커널은 입력 데이터의 채널 수와 동일한 채널 수를 가져야 한다.
- 이에 따라 하나의 커널이 가지는 매개변수의 수는 $K_i \times K_o \times C_i$이다.

- 그런데 이러한 커널이 총 $C_o$개가 있어야 하므로 가중치 매개변수의 총 수는 $K_i \times K_o \times C_i \times C_o$와 같다.

<br>

## 1.9 풀링 (Pooling)

- 일반적으로 합성곱 층(합성곱 연산 + 활성화 함수) 다음에는 풀링 층을 추가하는 것이 일반적이다.
- 풀링 층에서는 특성 맵을 **다운 샘플링**하여 특성 맵의 크기를 줄이는 풀링 연산이 이루어진다.
- 풀링 연산에는 일반적으로 **최대 풀링(max pooling)**과 **평균 풀링(average pooling)**이 사용된다.

<br>

### 1.9.1 풀링 연산의 이해

- 우선 최대 풀링을 통해서 풀링 연산을 이해해보자.

$\quad$ ![](https://wikidocs.net/images/page/64066/maxpooling.PNG)

- 풀링 연산에서도 합성곱 연산과 마찬가지로 커널과 스트라이드의 개념을 가진다.
- 위의 그림은 스트라이드가 2일 때, 2 x 2 크기의 커널로 맥스 풀링 연산을 했을 때 특성 맵이 절반의 크기로 다운샘플링되는 것을 보여준다.
- 맥스 풀링은 커널과 겹치는 영역 안에서 최대값을 추출하는 방식으로 다운샘플링한다.

- 다른 풀링 기법인 평균 풀링은 최대값을 추출하는 것이 아니라 평균값을 추출하는 연산이 된다.

<br>

### 1.9.2 풀링 연산과 합성곱 연산의 차이

- 풀링 연산은 커널과 스트라이드 개념이 존재한다는 점에서 합성곱 연산과 유사하다.
- 합성곱 연산과의 차이점은 다음 2가지이다.
  1. 학습해야 할 가중치가 없다.
  2. 연산 후에 채널 수가 변하지 않는다.

<br>

### 1.9.3 풀링의 특징

- 풀링을 사용하면, 특성 맵의 크기가 줄어드므로 특성 맵의 가중치의 개수를 줄여준다.

<br>

## 1.10 참고 링크

- [https://www.learnopencv.com/number-of-parameters-and-tensor-sizes-in-convolutional-neural-network/](https://www.learnopencv.com/number-of-parameters-and-tensor-sizes-in-convolutional-neural-network/)